In [ ]:
!git clone https://github.com/time-series-foundation-models/lag-llama/

fatal: destination path 'lag-llama' already exists and is not an empty directory.


In [ ]:
cd lag-llama

/content/lag-llama


In [ ]:
!pip install -r requirements.txt --quiet

In [ ]:
!huggingface-cli download time-series-foundation-models/Lag-Llama lag-llama.ckpt --local-dir /content/lag-llama

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
/content/lag-llama/lag-llama.ckpt


In [ ]:
!pip install lightning


In [ ]:
from itertools import islice

from matplotlib import pyplot as plt
import matplotlib.dates as mdates

import torch
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from gluonts.dataset.pandas import PandasDataset
import pandas as pd

from lag_llama.gluon.estimator import LagLlamaEstimator

/usr/local/lib/python3.10/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [ ]:
def get_lag_llama_predictions(dataset, prediction_length, num_samples=100):
    ckpt = torch.load("lag-llama.ckpt", map_location=torch.device('cuda:0')) # Uses GPU since in this Colab we use a GPU.
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=32, # Should not be changed; this is what the released Lag-Llama model was trained with

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],

        batch_size=1,
        num_parallel_samples=100
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(forecast_it)
    tss = list(ts_it)

    return forecasts, tss

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

url = ("/content/drive/MyDrive/final_dataset.csv")
df = pd.read_csv(url, index_col = 0 , parse_dates=True )
df.head()

,AQI_calculated
date,
2020-11-25 16:00:00,480.0
2020-11-25 17:00:00,482.0
2020-11-25 18:00:00,482.0
2020-11-25 19:00:00,481.0
2020-11-25 20:00:00,479.0


In [ ]:
df.index.name = None

In [ ]:
from gluonts.dataset.pandas import PandasDataset
import pandas
for col in df.columns:
    # Check if column is not of string type
    if df[col].dtype != 'object' and pd.api.types.is_string_dtype(df[col]) == False:
        df[col] = df[col].astype('float32')
ds = PandasDataset(df, target="AQI_calculated" , freq = "1H")
backtest_dataset = ds
prediction_length = 1 # Define your prediction length. We use 24 here since the data is of hourly frequency
num_samples = 24

In [ ]:
forecasts, tss = get_lag_llama_predictions(backtest_dataset, prediction_length, num_samples)

In [ ]:
len(forecasts)

1

In [ ]:
forecasts[0]

gluonts.model.forecast.SampleForecast(info=None, item_id=None, samples=array([[567.7641 ],
       [572.8478 ],
       [560.83997],
       [563.28186],
       [573.3078 ],
       [575.14484],
       [570.36505],
       [593.48456],
       [567.90704],
       [572.07166],
       [574.34344],
       [577.61664],
       [561.73425],
       [578.1479 ],
       [586.1107 ],
       [564.5761 ],
       [577.1028 ],
       [570.5899 ],
       [574.31433],
       [570.89166],
       [572.8419 ],
       [573.7318 ],
       [573.07056],
       [562.35864]], dtype=float32), start_date=Period('2023-01-24 08:00', 'H'))

In [ ]:
import pandas as pd

dfs = []
for forecast in forecasts:
    samples = forecast.samples
    start_date = forecast.start_date.to_timestamp()

    # Convert the samples array to a DataFrame
    df = pd.DataFrame(samples)

    # Add a column for the date index based on the start_date
    df.index = pd.date_range(start='2023-01-24 09:00', periods=len(df), freq='H')

    dfs.append(df)

# Concatenate the list of DataFrames into a single DataFrame
result_df = pd.concat(dfs)

# Display the concatenated DataFrame
print(result_df)


                              0
2023-01-24 09:00:00  567.764099
2023-01-24 10:00:00  572.847778
2023-01-24 11:00:00  560.839966
2023-01-24 12:00:00  563.281860
2023-01-24 13:00:00  573.307800
2023-01-24 14:00:00  575.144836
2023-01-24 15:00:00  570.365051
2023-01-24 16:00:00  593.484558
2023-01-24 17:00:00  567.907043
2023-01-24 18:00:00  572.071655
2023-01-24 19:00:00  574.343445
2023-01-24 20:00:00  577.616638
2023-01-24 21:00:00  561.734253
2023-01-24 22:00:00  578.147888
2023-01-24 23:00:00  586.110718
2023-01-25 00:00:00  564.576111
2023-01-25 01:00:00  577.102783
2023-01-25 02:00:00  570.589905
2023-01-25 03:00:00  574.314331
2023-01-25 04:00:00  570.891663
2023-01-25 05:00:00  572.841919
2023-01-25 06:00:00  573.731812
2023-01-25 07:00:00  573.070557
2023-01-25 08:00:00  562.358643
